In [53]:
## !/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: rohantondulkar,QwQ2000
"""

import numpy as np
import pandas as pd
from dateutil import parser
import time
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval
from gensim.models import Doc2Vec
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle
import re
from progressbar import *    
from pyltr.models.lambdamart import LambdaMART
from pyltr.util.group import check_qids, get_groups

class MyLambdaMART(LambdaMART):
     def _fit_stages(self, X, y, qids, y_pred, random_state,
                    begin_at_stage=0, monitor=None):
        n_samples = X.shape[0]
        do_subsample = self.subsample < 1.0
        sample_weight = np.ones(n_samples, dtype=np.float64)

        n_queries = check_qids(qids)
        query_groups = np.array([(qid, a, b, np.arange(a, b))
                                 for qid, a, b in get_groups(qids)],
                                dtype=np.object)
        assert n_queries == len(query_groups)
        do_query_oob = self.query_subsample < 1.0
        query_mask = np.ones(n_queries, dtype=np.bool)
        query_idx = np.arange(n_queries)
        q_inbag = max(1, int(self.query_subsample * n_queries))

        if self.verbose:
            verbose_reporter = _VerboseReporter(self.verbose)
            verbose_reporter.init(self, begin_at_stage)
        j = 0
        for i in range(begin_at_stage, self.n_estimators):
            j = i
            if do_query_oob:
                random_state.shuffle(query_idx)
                query_mask = np.zeros(n_queries, dtype=np.bool)
                query_mask[query_idx[:q_inbag]] = 1

            query_groups_to_use = query_groups[query_mask]
            sample_mask = np.zeros(n_samples, dtype=np.bool)
            for qid, a, b, sidx in query_groups_to_use:
                sidx_to_use = sidx
                if do_subsample:
                    query_samples_inbag = max(
                        1, int(self.subsample * (b - 1)))
                    random_state.shuffle(sidx)
                    sidx_to_use = sidx[:query_samples_inbag]
                sample_mask[sidx_to_use] = 1

            if do_query_oob:
                old_oob_total_score = 0.0
                for qid, a, b, _ in query_groups[~query_mask]:
                    old_oob_total_score += self.metric.evaluate_preds(
                        qid, y[a:b], y_pred[a:b])

            y_pred = self._fit_stage(i, X, y, qids, y_pred, sample_weight,
                                     sample_mask, query_groups_to_use,
                                     random_state)

            train_total_score, oob_total_score = 0.0, 0.0
            for qidx, (qid, a, b, _) in enumerate(query_groups):
                score = self.metric.evaluate_preds(
                    qid, y[a:b], y_pred[a:b])
                if query_mask[qidx]:
                    train_total_score += score
                else:
                    oob_total_score += score

            self.train_score_[i] = train_total_score / q_inbag
            if do_query_oob:
                if q_inbag < n_queries:
                    self.oob_improvement_[i] = \
                        ((oob_total_score - old_oob_total_score) /
                         (n_queries - q_inbag))

            early_stop = False
            monitor_output = None
            if monitor is not None:
                monitor_output = monitor(i, self, locals())
                if monitor_output is True:
                    early_stop = True

            if self.verbose > 0:
                verbose_reporter.update(i, self, monitor_output)

            if early_stop:
                break

        return j + 1
    
class BestAnswererPredictor:
    """ Model to make real time predictions of best answerer for new questions"""
    
    def __init__(self, train_profile, user_details, user_tag_profile, tags_list, user_text_vec, doc2vec_model, ltr_model ):
        # 2-d numpy array for all questions in the format
        # PostId, QuestionContent, AnswerId, AnswerContent, IsBestAnswer, Score, Tags, AnswererId, AnswerTime, TotalContent
        self.train_profile = train_profile
        
        # Get 2-D array of user details in the format 
        # UserId, AnswerFrequencyInDays, Reputation, NumBestAnsToAnsRatio, MRR, AvgQuesQuality, AvgQuesPopularity
        self.user_details = user_details
        self.user_list = self.user_details[:,0]
        self.num_users = len(self.user_list)

        # Get 2-D array for tag scores on 987 tags per user
        self.user_tag_profile = user_tag_profile
        self.tags_list = tags_list
        
        # Get 2-D array of text vector representation per user
        self.user_text_vec = user_text_vec
        
        self.doc2vec_model = doc2vec_model
        self.ltr_model = ltr_model
        self.stop_words = set(stopwords.words('english'))
        self.pstem = PorterStemmer()
        self.num_features = 9
        self.feature_set = [ self.get_tag_similarity, self.get_cosine_similarity, self.get_user_answer_frequency,
                            self.get_number_best_answers_ratio, self.get_user_reputation, self.get_days_to_prev_ans,
                            self.get_user_MRR, self.get_user_avg_ques_quality, self.get_user_avg_ques_popularity
                           ]
        self.setup()
        
    def setup(self):
        """ Prepare internal data for faster prediction """
        print('Setting up...')
        self.user_train_profiles = {}
        for user in self.user_list:
            self.user_train_profiles[user] = self.train_profile[np.where(self.train_profile[:,7]==user)]
        print('Setup finished!')
        
    def get_train_data(self,data_line,answerer = -1):
        """ Expecting a 1-d numpy array as [QuestionContent, Tags, CreateTime (datetime object)]"""
        # Create a feature set for all ques-user pairs
        features = np.zeros((len(self.user_list), self.num_features + 2))
        features[:, 0] = self.user_list
        
        labels = np.zeros(len(self.user_list),dtype = float)
        for i in range(len(self.user_list)):
            if features[i][0] == answerer:
                labels[i] = 1.0
                break
        
        # Expected order of feature set: 
        # ['SimilarityScore', 'CosineSimilarity', 'AnswerFrequencyInDays', 'NumBestAnsToAnsRatio', 
        #  'Reputation', 'DaysToPrevAns', 'MRR', 'AvgQuesQuality', 'AvgQuesPopularity']     
        for i in range(1,self.num_features):
            #start = time.time()
            features[:, i] = self.feature_set[i-1](data_line)
            #print('Time for feature: {0} is {1} secs'.format(self.feature_set[i-1],time.time()-start))
        return features,labels
    
    def predict(self, test_data, topK):
        """ Expecting a 1-d numpy array as [QuestionContent, Tags, CreateTime (datetime object)]"""
        features,_ = self.get_train_data(test_data)
        features[:,10] = self.ltr_model.predict(features[:,1:self.num_features+1])
        ranked_users = features[features[:,10].argsort()[::-1]][:,0]
        #print(ranked_users[:topK])
        return ranked_users.astype(int)[:topK]
        
    def get_cosine_similarity(self, test_data):
        """ Get cosine similarity between doc2vec representations of question and all users """
        text_score = np.zeros(self.num_users)
        text = test_data[0]
        text = re.compile('\w+').findall(text.lower())
        text = [w for w in text if not w in self.stop_words]
        text = [self.pstem.stem(w) for w in text if len(w)>2]
        text_vec = self.doc2vec_model.infer_vector(text)
        count = 0
        
        for user in self.user_list:
            text_score[count] = cosine_similarity(text_vec.reshape(1, -1), self.user_text_vec[self.user_text_vec[:,0]==user][:,1:])
            count+=1
        return text_score
        
    def get_tag_similarity(self, test_data):
        """ Get tag based similarity between tag profiles of question and all users """
        tags = literal_eval(test_data[1])
        ques_tag_vec = np.zeros(len(self.tags_list))
        
        # Make ques tag vector
        for tag in tags:
            res = np.where(self.tags_list == tag)
            if len(res[0]) != 0:
                i = res[0][0]
                ques_tag_vec[i]=1
            
        # Calculate tag score for all users
        count = 0
        tag_score = np.zeros(self.num_users)
        for user in self.user_list:
            user_tag_vec = self.user_tag_profile[np.where(self.user_tag_profile[:, 0]==user)][:,1:].ravel()
            match = user_tag_vec[np.logical_and(ques_tag_vec, user_tag_vec)]
            tag_score[count] = np.sum(match) * match.shape[0]
            count+=1
        return tag_score
    
    def get_days_to_prev_ans(self, test_data):
        """ Get the number of days since last answer for each user """
        days_to_prev_ans = np.zeros(self.num_users)
        create_time = parser.parse(test_data[2])
        count = 0
        
        for user in self.user_list:
            # Get the list of answers by the given userId
            ans_profile = self.user_train_profiles[user]

            # Get the days since previous answer
            ans_profile = ans_profile[np.where(ans_profile[:,8]<create_time)]

            if len(ans_profile) == 0:
                days_to_prev_ans[count] = 2000
            else:
                ans_profile = ans_profile[ans_profile[:,8].argsort()[::-1]]
                timeDiff = create_time - ans_profile[0, 8]
                days_to_prev_ans[count] = timeDiff.total_seconds()/(3600*24)        # in days
            count+=1
        return days_to_prev_ans
    
    def get_user_answer_frequency(self, test_data):
        """ Get answering frequeny for all users """
        return self.user_details[:,1]
    
    def get_user_reputation(self, test_data):
        """ Get reputation for all users """
        return self.user_details[:,2]
    
    def get_number_best_answers_ratio(self, test_data):
        """ Get number best answers to number of answers ratio"""
        return self.user_details[:,3]
    
    def get_user_MRR(self, test_data):
        """ Get mean reciprocal rank for every user """
        return self.user_details[:, 4]
    
    def get_user_avg_ques_quality(self, test_data):
        """ Get average question quality per user"""
        return self.user_details[:, 5]
    
    def get_user_avg_ques_popularity(self, test_data):
        """ Get average question popularty per user"""
        return self.user_details[:, 6]
    

def sample_predictions():
    """ An example to show how to make sample predictions"""
    # All features should be only for 1339 final set of users
    # Load all the files needed
    doc2vecModel = Doc2Vec.load('../Models/user_doc2vec_1000.model')
    user_details = pd.read_csv('../Dataset/UserInformation.csv')
    train_profile = pd.read_csv('../Dataset/Train_Profile.csv')
    train_profile['AnswerTime'] = pd.to_datetime(train_profile['AnswerTime'])
    test_ltr = pd.read_csv('../Dataset/Sample_test_data.csv')
    user_tag_profile = pd.read_csv('../Dataset/User_tag_profile.csv')
    user_text_vec = pd.read_csv('../Dataset/User_text_vec.csv')

    with open(r"../Models/LTR_ALL_final.pkl", "rb") as input_file:
        ltr_model = pickle.load(input_file)
        
    # Initialize the model
    predictor = BestAnswererPredictor(train_profile = train_profile.values, user_details = user_details.values, \
                                  user_tag_profile = user_tag_profile.values, tags_list = user_tag_profile.columns[1:],\
                                  user_text_vec = user_text_vec.values,\
                                  doc2vec_model = doc2vecModel, ltr_model = ltr_model)
    topK = 50
    for index, row in test_ltr.iterrows():
        start = time.time()
        topK_users = predictor.predict(row.values, topK)
        print(index)
        print('\nPrediction took: {0} secs'.format(time.time()-start))
        print('Top {0} users: {1}'.format(topK, topK_users))

def get_test_metrics(model_path):
    """ Show ACC and MRR on test set"""
    doc2vecModel = Doc2Vec.load('../Models/user_doc2vec_1000.model')
    user_details = pd.read_csv('../Dataset/UserInformation.csv')
    train_profile = pd.read_csv('../Dataset/Train_Profile.csv')
    train_profile['AnswerTime'] = pd.to_datetime(train_profile['AnswerTime'])
    test_ltr = pd.read_csv('../Dataset/Test_LTR.csv')
    user_tag_profile = pd.read_csv('../Dataset/User_tag_profile.csv')
    user_text_vec = pd.read_csv('../Dataset/User_text_vec.csv')

    with open(model_path, "rb") as input_file:
        ltr_model = pickle.load(input_file)
        
    # Initialize the model
    predictor = BestAnswererPredictor(train_profile = train_profile.values, user_details = user_details.values, \
                                  user_tag_profile = user_tag_profile.values, tags_list = user_tag_profile.columns[1:],\
                                  user_text_vec = user_text_vec.values,\
                                  doc2vec_model = doc2vecModel, ltr_model = ltr_model)
    topK = [1,5,10,20,50]
    acc,mrr = [0 for i in range(len(topK))],[0 for i in range(len(topK))]
    start = time.time()
    print('Start calculating metrics...')
    bar = ProgressBar(maxval = len(test_ltr)).start()
    for index, row in test_ltr.iterrows():
        topK_users = predictor.predict([row.values[1],row.values[6],row.values[8]], topK[-1])
        best = row.values[7]
        bestPos = topK[-1] + 1
        for j in range(topK[-1]):
            if topK_users[j] == best:
                bestPos = j
                break
        for j in range(len(topK)):
            if bestPos < topK[j]:
                acc[j] += 1
                mrr[j] += 1 / (bestPos + 1)
        if index % 99 == 0 or index + 1 == len(test_ltr):
            bar.update(index + 1)
            print('\nPrediction took: {0} secs'.format(time.time()-start))
            print('ACC: {0} \nMRR: {1}'.format([v / (index + 1) for v in acc],[v / (index + 1) for v in mrr]))

def train_ltr_model(model_path,output_path):
    """Train LambdaMART model."""
    doc2vecModel = Doc2Vec.load('../Models/user_doc2vec_1000.model')
    user_details = pd.read_csv('../Dataset/UserInformation.csv')
    train_profile = pd.read_csv('../Dataset/Train_Profile.csv')
    train_profile['AnswerTime'] = pd.to_datetime(train_profile['AnswerTime'])
    train_ltr = pd.read_csv('../Dataset/Train_LTR.csv')
    user_tag_profile = pd.read_csv('../Dataset/User_tag_profile.csv')
    user_text_vec = pd.read_csv('../Dataset/User_text_vec.csv')
    
    predictor = BestAnswererPredictor(train_profile = train_profile.values, user_details = user_details.values, \
                                  user_tag_profile = user_tag_profile.values, tags_list = user_tag_profile.columns[1:],\
                                  user_text_vec = user_text_vec.values,\
                                  doc2vec_model = doc2vecModel, ltr_model = None)
    
    x,y,qids = None,None,[]
    div = 500
    for index,row in train_ltr.iterrows():
        if index % div != 0:
            continue
        x0,y0 = predictor.get_train_data([row.values[1],row.values[6],row.values[8]],row.values[7])
        x = x0 if x is None else np.concatenate((x,x0))
        y = y0 if y is None else np.concatenate((y,y0))
        qids += [row.values[0] for i in range(len(predictor.user_list))]
    print('Feature ready.')
    with open(model_path, "rb") as input_file:
        model = MyLambdaMART()
    print('Fitting...')
    model.fit(x,y,qids)
    print(model.feature_importances_)
    print('Saving model...')
    with open(output_path,'wb') as output_file:
        pickle.dump(model,output_file)
    
if __name__ == "__main__":
    #sample_predictions()
    #train_ltr_model(r"../Models/LTR_ALL_final.pkl",r"../Models/LTR_PART.pkl")
    #get_test_metrics(r"../Models/LTR_ALL_final.pkl")
    get_test_metrics(r"../Models/LTR_PART.pkl")

Setting up...


Setup finished!
Start calculating metrics...

Prediction took: 0.8316562175750732 secs
ACC: [0.0, 1.0, 1.0, 1.0, 1.0] 
MRR: [0.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]



Prediction took: 117.39901232719421 secs
ACC: [0.04, 0.11, 0.19, 0.38, 0.67] 
MRR: [0.04, 0.06566666666666666, 0.07510714285714286, 0.08775451608075442, 0.09698278541388468]



Prediction took: 226.1710183620453 secs
ACC: [0.020100502512562814, 0.08542713567839195, 0.15577889447236182, 0.34673366834170855, 0.678391959798995] 
MRR: [0.020100502512562814, 0.043383584589614735, 0.051818616893993766, 0.06453386374003389, 0.0748190706109348]



Prediction took: 354.0906386375427 secs
ACC: [0.02348993288590604, 0.09731543624161074, 0.17114093959731544, 0.35906040268456374, 0.6610738255033557] 
MRR: [0.02348993288590604, 0.048601789709172255, 0.0573186321508469, 0.06969659336457079, 0.07913184000255787]


...


Prediction took: 3810.1741740703583 secs
ACC: [0.01734357848518112, 0.08869374313940724, 0.1743139407244786, 0.3398463227222832, 0.6577387486278814] 
MRR: [0.01734357848518112, 0.04050493962678372, 0.05144277176659161, 0.06271461718760456, 0.07283681658385896]



Prediction took: 3913.2901401519775 secs
ACC: [0.016974645466265578, 0.08766652342071336, 0.17318435754189945, 0.33820369574559517, 0.6568543188654921] 
MRR: [0.016974645466265578, 0.03994413407821225, 0.0508772961985251, 0.06212567076261993, 0.07228239771771061]



Prediction took: 4034.5292711257935 secs
ACC: [0.016831474857984433, 0.08689248895434462, 0.17273301073006522, 0.33705028403113824, 0.657269093204292] 
MRR: [0.016831474857984433, 0.03953993968721505, 0.0505185530274946, 0.061714059107818395, 0.07193299954844153]



Prediction took: 4148.9869611263275 secs
ACC: [0.016488046166529265, 0.08635614179719703, 0.1718878812860676, 0.3365622423742787, 0.6589035449299258] 
MRR: [0.016488046166529265, 0.03920376477054132, 0.05013061215142816, 0.061345120523814335, 0.07162057010647256]



Prediction took: 4262.6138434410095 secs
ACC: [0.016380182002022244, 0.08574317492416582, 0.1716885743174924, 0.3356926188068756, 0.6590495449949444] 
MRR: [0.016380182002022244, 0.03897876643073808, 0.049984993660423235, 0.0611544782132468, 0.07144793716954363]


In [29]:
import pickle
import pyltr

with open(r"../Models/LTR_ALL_final.pkl", "rb") as input_file:
    ltr_model = pickle.load(input_file)
    for attr in dir(ltr_model):
        print(attr,getattr(ltr_model,attr))

__class__ <class '__main__.MyLambdaMART'>
__delattr__ <method-wrapper '__delattr__' of MyLambdaMART object at 0x00000270848A9358>
__dict__ {'metric': <pyltr.metrics.dcg.NDCG object at 0x00000270848A9080>, 'learning_rate': 0.1, 'n_estimators': 100, 'query_subsample': 1.0, 'subsample': 1.0, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 3, 'random_state': None, 'max_features': None, 'verbose': 0, 'max_leaf_nodes': None, 'warm_start': True, 'n_features': 11, 'max_features_': 11, 'estimators_': array([[DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=True,
           random_state=<mtrand.RandomState object at 0x0000027088C93510>,
           splitter='best')],
       [DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
           max_features=Non